## Librerie

In [1]:
from scipy.optimize import minimize, basinhopping
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

# Range ammissibile percentuale materiale per ricetta
#           | Materiale1        | Materiale2 | ...
# ------------------------------------------------
# Ricetta1  | (val att. ,range) |            |
# Ricetta2  |                   |            |
# ...
range_ric_mat = np.array([
    [(0.58, 0.01), (0.396, 0.003), (0.024, 0.001)], # ricetta 0
    [(0.625, 0.005), (None, None), (None, None)],
    [(0.62, 0.01), (None, None), (None, None)],
])

In [3]:
%run ./importazione.ipynb

(10,)
(28,)
(28, 10)
(28, 13)
(10, 13)
(10, 13)
(10, 13)


In [4]:
print('Verifica dimensioni matrici')
print(f'Consumi (num. famiglie)): {consumi.shape}')
print(f'Produzioni (num. ricette): {produzioni.shape}')
print(f'Ricette (famiglie x ricette): {ricette.shape}')
print(f'Composizioni (famiglie x materiali): {composizioni_famiglia.shape}')
print(f'Range (ricette x materiali): {range_ric_mat.shape}')

Verifica dimensioni matrici
Consumi (num. famiglie)): (28,)
Produzioni (num. ricette): (10,)
Ricette (famiglie x ricette): (280,)
Composizioni (famiglie x materiali): (364,)
Range (ricette x materiali): (10, 13)


## Accrocchio per compattare ricetta

In [5]:
POSITIONS_COMP = []
RICETTA_COMP = []

for index, value in enumerate(ricette):
    if value != 0:
        POSITIONS_COMP.append(index)
        RICETTA_COMP.append(value)

def rebuild_ricetta(ricetta_comp):
    rebuild = np.zeros(len(ricette))
    for index, pos in enumerate(POSITIONS_COMP):
        rebuild[pos] = ricetta_comp[index]
    return rebuild

print(rebuild_ricetta(RICETTA_COMP)[0:20])
print(ricette[0:20])
print(len(RICETTA_COMP))

[0.    0.029 0.    0.    0.    0.    0.    0.    0.    0.    0.455 0.48
 0.03  0.475 0.    0.    0.    0.    0.005 0.   ]
[0.    0.029 0.    0.    0.    0.    0.    0.    0.    0.    0.455 0.48
 0.03  0.475 0.    0.    0.    0.    0.005 0.   ]
76


## Calcolo resa globale

In [6]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0389346559077535)'

## Funzioni di calcolo

In [7]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    ricetta = rebuild_ricetta(ricetta) #REBUILD
    return ricetta.reshape(-1, len(produzioni)) * produzioni #/ resa_globale

In [8]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [9]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sqrt(np.sum(np.square(tot_consumi-consumi)))/1000
    return tot_err

In [10]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [11]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

In [12]:
# Percentuali di ciascun materiale in una ricetta
def perc_mat(ricetta, id_ric):
    cons_fam = calc_mat_consumi(ricetta)
    compos = composizioni_famiglia.reshape(len(consumi), -1)
    cons_ricetta0 = np.vstack(cons_fam[:,id_ric])
    cons_materiali_ricetta0 = cons_ricetta0 * compos
    tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
    tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0) if np.sum(tot_cons_ricetta0) != 0 else np.zeros((len(tot_cons_ricetta0),1))
    return tot_perc_ricetta0

In [13]:
# Funzione errore percentuale materiale (obiettivo: >=0)
def err_perc_mat(ricetta, id_ric, id_mat, expected_val, expected_error):
    return expected_error - np.abs(perc_mat(ricetta, id_ric=id_ric)[id_mat] - expected_val)

## Ottimizzazione

In [88]:
%%time

constraints = [
    {'type': 'eq', 'fun': calc_err_totali},
]

for id_ric, ric in enumerate(range_ric_mat):
    for id_mat, mat in enumerate(ric):
        if not np.isnan(mat[0]):
            constr = {'type': 'ineq', 'fun': err_perc_mat, 'args': (id_ric, id_mat, mat[0], mat[1])}
            constraints.append(constr)

# constraints = (
#     {'type': 'eq', 'fun': calc_err_totali},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 0, 0.58, 0.01)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 1, 0.396, 0.003)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 2, 0.024, 0.001)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (1, 0, 0.625, 0.005)},
#     # {'type': 'ineq', 'fun': err_perc_mat, 'args': (2, 0, 0.62, 0.01)},
# )

#DEBUG
print(len(constraints))
# constraints = constraints[0:40] #DEBUG 40 ok, oltre boom

bounds = list(( (0, 1) for x in range(len(RICETTA_COMP) )))

# Forza a zero i valori che nella ricetta originale sono già nulli
# for index, value in enumerate(ricette):
#     if value == 0:
#         bounds[index] = (0,0)

count_iterations = 0
def callback(x, y):
    global count_iterations
    print(f'{count_iterations} - {calc_err_totali(x)}')
    count_iterations += 1

res = minimize(
    calc_error_resa, 
    RICETTA_COMP, 
    method='trust-constr', 
    # method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    callback=callback,
    options={'disp': True, 'maxiter':100}
)

#DEBUG
# res.x = RICETTA_COMP

res.x = res.x / resa_globale

print(res)

# res = basinhopping(
#     calc_error_resa, 
#     RICETTA_COMP,
#     stepsize=0.05,
#     niter=10,
    
#     minimizer_kwargs = {
#         "method":'SLSQP',
#         "constraints":constraints,
#         "bounds":bounds,
#         "callback":callback,
#         "options":{'disp': True, 'maxiter':20}
#     }
# )
# print(res)

# for i in range(10):
    
#     res = minimize(
#         calc_error_resa, 
#         res.x, 
#         method='trust-constr', 
#         # method='SLSQP',
#         constraints=constraints,
#         bounds=bounds,
#         options={'disp': True, 'maxiter':10}
#     )
        
#     res = minimize(
#         calc_error_resa, 
#         res.x,
#         method='SLSQP',
#         constraints=constraints,
#         bounds=bounds,
#         callback=callback,
#         options={'disp': True, 'maxiter':10}
#     )

# print(res)


131
0 - 18234.605238100565


C:\Users\mario.piccinelli\Documents\Progetti\Almag-Spalmatura_prod\.venv\Lib\site-packages\scipy\optimize\_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


1 - 3290.3678459739567
2 - 3290.3678459739567
3 - 3290.3678459739567
4 - 3290.3678459739567
5 - 3290.3678459739567
6 - 3290.3678459739567
7 - 3013.53162991611
8 - 3013.53162991611
9 - 2984.4093250154106
10 - 2984.4093250154106
11 - 2928.2260834218114
12 - 2920.3630350757217
13 - 2920.3630350757217
14 - 2844.6454469258288
15 - 2790.5363726016285
16 - 2790.5363726016285
17 - 2790.5363726016285
18 - 2755.3086541474026
19 - 2748.2535654314806
20 - 2740.2453666725323
21 - 2718.1430779770344
22 - 2711.4663399879446
23 - 2687.817528490511
24 - 2682.9293342330675
25 - 2657.1230051489447
26 - 2653.2752825014923
27 - 2622.2024568668853
28 - 2619.672388330477
29 - 2590.3434484501513
30 - 2587.750240675461
31 - 2559.9525667687085
32 - 2555.9786968261465
33 - 2529.256698850618
34 - 2526.6721595479903
35 - 2495.8317294982244
36 - 2495.8317294982244
37 - 2412.2627338650836
38 - 2412.2627338650836
39 - 2396.8028454136843
40 - 2383.089729895069
41 - 2365.36233426006
42 - 2353.994263746779
43 - 2335.579

In [89]:
# Esperimento
# Proviamo a escludere un constraint alla volta e vediamo se troviamo una situazione
# in cui il problema converge.

# for i in range(1, len(constraints)):
    
#     print(i)
    
#     const_red = constraints[0:i] + constraints[i+1:]

#     res = minimize(
#         calc_error_resa, 
#         ricette, 
#         method='SLSQP',
#         constraints=const_red,
#         bounds=bounds,
#         options={'disp': False, 'maxiter':100}
#     )

#     print(f'{res.success} - {res.message} ({res.nit} iterations)')


## Verifiche

In [90]:
pd.options.display.float_format = '{:.2f}'.format

In [91]:
# res.x = res.x / resa_globale
result = rebuild_ricetta(res.x)

In [92]:
print("Percentuali aggiustate (in %)")
print(np.sum(result.reshape(len(consumi), len(produzioni))*100, axis=0))
pd.DataFrame(
    result.reshape(len(consumi), len(produzioni))*100
)

Percentuali aggiustate (in %)
[100.53941092 100.43924551 101.29103693 101.64355232 101.20319321
 101.40445406 101.06466832 101.37842905 100.78997162 101.24527074]


,0,1,2,3,4,5,6,7,8,9
0,0.00,3.63,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,45.34,51.04,6.32,54.27,0.00,0.00,0.00,0.00,5.86,0.00
2,0.00,0.00,35.64,0.00,7.75,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,20.41,0.00,0.00,0.00,28.19,0.00
4,0.00,0.00,0.00,0.00,0.00,44.76,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.77,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.16
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.42,0.00
8,27.37,23.77,5.72,31.54,0.00,0.00,0.00,0.00,5.63,0.00
9,0.00,0.00,27.51,0.00,5.00,3.06,0.00,0.00,0.00,0.00


In [93]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

Matrice consumi


,0,1,2,3,4,5,6,7,8,9
0,0.00,1271069.36,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2629026.98,17859507.65,472637.95,50610556.71,0.00,0.00,0.00,0.00,293575.00,0.00
2,0.00,0.00,2666185.97,0.00,290828.75,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,765749.81,0.00,0.00,0.00,1413422.29,0.00
4,0.00,0.00,0.00,0.00,0.00,608125.07,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,545987.86,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9035.65
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,422094.63,0.00
8,1587204.28,8318840.20,428202.23,29406847.19,0.00,0.00,0.00,0.00,282456.93,0.00
9,0.00,0.00,2057757.40,0.00,187662.97,41586.05,0.00,0.00,0.00,0.00


In [94]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

Verifica totale consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1271069.36,71865304.28,2957014.71,2179172.09,608125.07,545987.86,9035.65,422094.63,40023550.83,2287006.42,...,10041334.09,478398.02,270679.62,13474.11,32756.07,65496.63,140673.87,31913.14,310720.07,33818.03


In [95]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x))*resa_globale - consumi])

Verifica errore consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,73176.01,72067.45,-75782.81,656370.81,561538.71,-151031.59,-14239.55,102308.74,47794.67,-18838.77,...,22082.98,-989202.72,223336.44,11648.72,29589.42,55841.72,121823.96,32942.67,316422.35,24291.72


In [96]:
# np.sum(calc_mat_consumi(res.x)*resa_globale, axis=1) / consumi * 100

In [97]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

Verifica rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,1.01,1.00,1.01,1.02,1.01,1.01,1.01,1.01,1.01,1.01


In [98]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

Verifica errore rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,-0.03,-0.03,-0.03,-0.02,-0.03,-0.02,-0.03,-0.03,-0.03,-0.03


### Errori percentuali materiali

In [99]:
def print_err_mat(id_ricetta, ricetta):
    return pd.DataFrame([
        perc_mat(RICETTA_COMP, id_ricetta) * 100,
        map(lambda x: x[0]*100, list(range_ric_mat[id_ricetta])),
        perc_mat(ricetta, id_ricetta) * 100
    ])

In [100]:
print('Percentuali materiali ricetta 0')
print_err_mat(0, res.x)

Percentuali materiali ricetta 0


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,55.65,13.37,0.12,0.14,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,30.67
1,59.40,2.05,0.16,0.15,0.05,0.02,0.01,0.00,0.01,0.01,0.00,0.00,38.13
2,59.66,5.71,0.12,0.14,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.32


In [101]:
print('Percentuali materiali ricetta 1')
print_err_mat(1, res.x)

Percentuali materiali ricetta 1


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,58.11,11.74,0.14,0.16,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,29.79
1,58.00,2.90,0.16,0.16,0.10,0.02,0.01,0.00,0.01,0.01,0.00,0.00,38.62
2,58.03,6.27,0.15,0.16,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,35.32


In [102]:
print('Percentuali materiali ricetta 2')
print_err_mat(2, res.x)

Percentuali materiali ricetta 2


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,64.22,0.81,0.08,0.07,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,34.79
1,61.80,2.00,0.06,0.06,0.05,0.02,0.01,0.00,0.10,0.01,0.00,0.00,35.88
2,63.04,0.87,0.08,0.08,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,35.90


### Funzioni errore

In [103]:
for index,c in enumerate(constraints):
    # print(c)
    if 'args' in c:
        print(f"{index} - {c['fun'](res.x, *c['args']):.4f} (ric:{c['args'][0]} mat:{c['args'][1]})")
    else:
        print(c['fun'](res.x))

3578.2063824614866
1 - 0.0074 (ric:0 mat:0)
2 - -0.0266 (ric:0 mat:1)
3 - 0.0096 (ric:0 mat:2)
4 - 0.0099 (ric:0 mat:3)
5 - 0.0099 (ric:0 mat:4)
6 - 0.0098 (ric:0 mat:5)
7 - 0.0099 (ric:0 mat:6)
8 - 0.0099 (ric:0 mat:7)
9 - 0.0099 (ric:0 mat:8)
10 - 0.0099 (ric:0 mat:9)
11 - 0.0099 (ric:0 mat:10)
12 - 0.0100 (ric:0 mat:11)
13 - -0.0281 (ric:0 mat:12)
14 - 0.0097 (ric:1 mat:0)
15 - -0.0237 (ric:1 mat:1)
16 - 0.0099 (ric:1 mat:2)
17 - 0.0100 (ric:1 mat:3)
18 - 0.0096 (ric:1 mat:4)
19 - 0.0099 (ric:1 mat:5)
20 - 0.0099 (ric:1 mat:6)
21 - 0.0100 (ric:1 mat:7)
22 - 0.0099 (ric:1 mat:8)
23 - 0.0099 (ric:1 mat:9)
24 - 0.0100 (ric:1 mat:10)
25 - 0.0100 (ric:1 mat:11)
26 - -0.0230 (ric:1 mat:12)
27 - -0.0024 (ric:2 mat:0)
28 - -0.0013 (ric:2 mat:1)
29 - 0.0098 (ric:2 mat:2)
30 - 0.0098 (ric:2 mat:3)
31 - 0.0097 (ric:2 mat:4)
32 - 0.0099 (ric:2 mat:5)
33 - 0.0099 (ric:2 mat:6)
34 - 0.0100 (ric:2 mat:7)
35 - 0.0090 (ric:2 mat:8)
36 - 0.0099 (ric:2 mat:9)
37 - 0.0100 (ric:2 mat:10)
38 - 0.0100 (ri

# ?

In [104]:
r = rebuild_ricetta(res.x).reshape(len(consumi), len(produzioni))
print(np.sum(r, axis=0))
print(np.sum(r*produzioni, axis=1))
print(np.sum(r*produzioni, axis=1) - consumi)
print(np.sum(np.sum(r*produzioni, axis=1) - consumi))

[1.00539411 1.00439246 1.01291037 1.01643552 1.01203193 1.01404454
 1.01064668 1.01378429 1.00789972 1.01245271]
[1.27106936e+06 7.18653043e+07 2.95701471e+06 2.17917209e+06
 6.08125066e+05 5.45987858e+05 9.03565485e+03 4.22094628e+05
 4.00235508e+07 2.28700642e+06 2.16807525e+06 5.12302468e+05
 5.28882843e+05 9.04165386e+03 2.36696394e+06 1.10869972e+07
 4.00361662e+05 4.98836377e+06 1.00413341e+07 4.78398016e+05
 2.70679623e+05 1.34741078e+04 3.27560700e+04 6.54966278e+04
 1.40673872e+05 3.19131444e+04 3.10720068e+05 3.38180267e+04]
[23687.357660785085 -2725983.44517456 -190913.15964504797
 571525.4941467517 537861.5661294144 -172289.44219913543
 -14591.34514878151 85874.62790042878 -1510508.5077872723
 -107882.58210133249 748842.947256177 27203.867943722813 189995.7428107272
 -18417.04614202003 -51326.058658952825 -667279.8000281062
 16606.662271563953 -185609.23362607416 -368872.91236349195
 -1007828.9836657499 212797.62322898518 11124.107760300327
 28314.070023017706 53291.6277505

In [105]:
89610471.44182177*(1-0.96)

3584418.857672874